In [1]:
import pandas as pd
import numpy as np

# H0 anisotropy

In [27]:
def H0_dipole_variation(scan_bf, scan_bt, n, A_all, A_all_plus, A_all_minus):
    """
    Arguments
    ---------
    scan_bf : pandas.DataFrame
        Best fit values of the scan.
    scan_bt : pandas.DataFrame
        Bootstrap values of the scan.
    n : float
        H0/H0_all = (A/A_all)^n.
    A_all : float
        Best fit value of A for all clusters.
    A_all_plus : float
        The positive 1-sigma uncertainty of A_all.
    A_all_minus : float
        The negative 1-sigma uncertainty of A_all. Here input as a positive number.
    """
    assert A_all_minus > 0
    assert A_all_plus > 0

    maxidx = scan_bf['n_sigma'].idxmax()
    minidx = scan_bf['n_sigma'].idxmin()

    A_all_sigma = np.max([A_all_plus, A_all_minus])

    # The + region
    Glon = scan_bf.loc[maxidx, 'Glon']
    Glat = scan_bf.loc[maxidx, 'Glat']

    A_max = scan_bf.loc[maxidx, 'A']
    A_max_distr = scan_bt[(scan_bt['Glon']==Glon) & (scan_bt['Glat']==Glat)]['A']
    A_max_distr = np.array(A_max_distr)

    # A_max_plus = np.percentile(A_max_distr, 84) - A_max
    # A_max_minus = A_max - np.percentile(A_max_distr, 16)
    # A_max_sigma = np.max([A_max_plus, A_max_minus])

    A_max_sigma = np.std(A_max_distr) # Use the standard deviation


    f_max = best_fit_H0_H0all_max = (A_max/A_all)**n
    H0_H0all_sigma_max = np.sqrt(n**2 * f_max**(2*n-2) / A_all**2 * A_max_sigma**2
                              + n**2 * f_max**(2*n-2) * A_max**2 / A_all**4 * A_all_sigma**2)
    
    # The - region
    Glon = scan_bf.loc[minidx, 'Glon']
    Glat = scan_bf.loc[minidx, 'Glat']

    A_min = scan_bf.loc[minidx, 'A']
    A_min_distr = scan_bt[(scan_bt['Glon']==Glon) & (scan_bt['Glat']==Glat)]['A']
    A_min_distr = np.array(A_min_distr)

    # A_min_plus = np.percentile(A_min_distr, 84) - A_min
    # A_min_minus = A_min - np.percentile(A_min_distr, 16)
    # A_min_sigma = np.max([A_min_plus, A_min_minus])

    A_min_sigma = np.std(A_min_distr) # Use the standard deviation

    f_min = best_fit_H0_H0all_min = (A_min/A_all)**n
    H0_H0all_sigma_min = np.sqrt(n**2 * f_min**(2*n-2) / A_all**2 * A_min_sigma**2
                              + n**2 * f_min**(2*n-2) * A_min**2 / A_all**4 * A_all_sigma**2)
    
    H0_H0all_sigma = np.sqrt(H0_H0all_sigma_max**2 + H0_H0all_sigma_min**2)
    return f_max-f_min, H0_H0all_sigma

In [28]:
scan_bf = pd.read_csv('../data/fits/scan_best_fit_LX-T_θ75.csv')
scan_bt = pd.read_csv('../data/fits/scan_btstrp_LX-T_θ75.csv')
H0_dipole_variation(scan_bf, scan_bt, 0.5, 1.514, 0.021 ,0.028)

(0.04838244768862321, 0.023225335104064493)

In [29]:
scan_bf = pd.read_csv('../data/fits/scan_best_fit_YSZ-T_θ60.csv')
scan_bt = pd.read_csv('../data/fits/scan_btstrp_YSZ-T_θ60.csv')
H0_dipole_variation(scan_bf, scan_bt, 0.5, 1.079, 0.018, 0.012)

(0.09843794296185082, 0.03209427308937998)

In [30]:
scan_bf = pd.read_csv('../data/fits/scan_best_fit_M-T_θ75.csv')
scan_bt = pd.read_csv('../data/fits/scan_btstrp_M-T_θ75.csv')
H0_dipole_variation(scan_bf, scan_bt, 0.4, 1.069, 0.012, 0.012)

(0.030507627422421968, 0.011531243814012231)

# Dipole direction uncertainty

In [2]:
scan_bf = pd.read_csv('../data/fits/scan_best_fit_LX-T_θ75.csv')

In [4]:
max_sigma = scan_bf['n_sigma'].max()
scan_bf.loc[(np.abs(scan_bf['n_sigma']) >= max_sigma-1), 'Glon']

60     -180.0
62     -180.0
67     -180.0
152    -176.0
153    -176.0
        ...  
7323    144.0
7987    172.0
8071    176.0
8074    176.0
8075    176.0
Name: Glon, Length: 1846, dtype: float64